In [ ]:
# PBS antibiotics (2015) + ABS ERP — my simple end-to-end script
# What I’m doing:
# 1) Clean the two raw CSVs (ABS population + PBS).
# 2) Compute scripts per 1,000 people (using ABS ERP for 2015).
# 3) Make 3 plots: (a) monthly average, (b) top 5 classes, (c) cumulative.

from pathlib import Path
import re
import pandas as pd
import matplotlib.pyplot as plt

# ---------- 0) Paths (I set this to where my raw files are) ----------
DATA_DIR = Path("/Users/sepidehmianji/Personal data projects for Linkedin/Project 1/pbs_antibiotics_project/data/raw")
ABS_IN = DATA_DIR / "population_sample.csv"
PBS_IN = DATA_DIR / "pbs-atc-2015.csv"

OUT_DIR = DATA_DIR / "cleaned"; OUT_DIR.mkdir(parents=True, exist_ok=True)
FIG_DIR = DATA_DIR / "figures"; FIG_DIR.mkdir(parents=True, exist_ok=True)

# ---------- small helpers I reuse ----------
def parse_month(series: pd.Series) -> pd.Series:
    """I try several common date formats for month labels."""
    s = series.astype(str).str.strip()
    p = pd.to_datetime(s, format="%b-%Y", errors="coerce")       # e.g., Dec-2015
    if p.notna().sum() == 0:
        p = pd.to_datetime(s, format="%B %Y", errors="coerce")   # e.g., December 2015
    if p.notna().sum() == 0:
        p = pd.to_datetime(s, errors="coerce")                   # fallback (YYYY-MM, etc.)
    return p

def to_number(x):
    """Turn strings like '12,345' into numbers, else NaN."""
    return pd.to_numeric(str(x).replace(",", ""), errors="coerce")

# =========================
# 1) Clean ABS population
# =========================
# The ABS CSV has multiple header bands + odd delimiters, so I let pandas sniff the delimiter.
abs_raw = pd.read_csv(ABS_IN, sep=None, engine="python", header=None,
                      encoding="latin1", on_bad_lines="skip")

# I find a cell that labels the month column (e.g., 'Collection Month' or 'Month').
month_pos = None
for i in range(min(250, len(abs_raw))):
    for j in range(min(100, abs_raw.shape[1])):
        cell = str(abs_raw.iat[i, j]).strip().lower()
        if cell.startswith("collection month") or cell == "month":
            month_pos = (i, j); break
    if month_pos: break
if not month_pos:
    raise SystemExit("ABS: couldn't find a 'Collection Month'/'Month' label in the file.")

mrow, mcol = month_pos

# I look for a nearby header row that mentions 'Estimated Resident Population' and 'Australia'.
erp_row = None
for i in range(max(0, mrow-60), min(abs_raw.shape[0], mrow+80)):
    text = " | ".join(abs_raw.iloc[i].astype(str)).lower()
    if ("estimated resident population" in text) and ("australia" in text):
        erp_row = i; break
if erp_row is None:
    raise SystemExit("ABS: couldn't find a header row for 'Estimated Resident Population' (Australia).")

# From the month column, I find where the real monthly data starts (first parseable date).
parsed_months = parse_month(abs_raw.iloc[mrow+1:, mcol])
if parsed_months.notna().sum() == 0:
    raise SystemExit("ABS: month values under the label aren’t parseable. Please open the CSV and check.")
data_start = parsed_months.dropna().index[0]

# I figure out which column contains the Australia ERP series by scanning the header band.
erp_col_idx = None
for j in range(abs_raw.shape[1]):
    col_text = " | ".join(abs_raw.iloc[erp_row:erp_row+3, j].astype(str)).lower()
    if ("estimated resident population" in col_text) and ("australia" in col_text):
        erp_col_idx = j; break
if erp_col_idx is None:
    raise SystemExit("ABS: couldn’t identify the ERP Australia column.")

# I build a tidy ABS dataframe with Month + ERP_Australia.
abs_months = parse_month(abs_raw.iloc[data_start:, mcol])
abs_erp = abs_raw.iloc[data_start:, erp_col_idx].map(to_number)
abs_clean = pd.DataFrame({"Month": abs_months, "ERP_Australia": abs_erp}).dropna().reset_index(drop=True)
abs_clean.to_csv(OUT_DIR / "population_clean.csv", index=False)

# I take the last available ERP value in 2015 (often December). If ABS reported in thousands, I scale up.
erp_2015 = abs_clean.loc[abs_clean["Month"].dt.year == 2015, "ERP_Australia"].iloc[-1]
if erp_2015 < 1_000_000:   # e.g., 23,940 means 23,940,000
    erp_2015 *= 1_000

# ======================
# 2) Clean PBS (2015)
# ======================
# PBS files come in a couple of schemas, so I handle both.
try:
    pbs_raw = pd.read_csv(PBS_IN, encoding="latin1")
except Exception:
    pbs_raw = pd.read_csv(PBS_IN, sep=None, engine="python", encoding="latin1", on_bad_lines="skip")

cols = set(pbs_raw.columns)

# I pick a services column robustly (Services / Number of services / Scripts).
svc_candidates = [c for c in pbs_raw.columns if c.lower().strip() in {"services", "number of services", "scripts"}]
if not svc_candidates:
    svc_candidates = [c for c in pbs_raw.columns if ("service" in c.lower()) or ("script" in c.lower())]
if not svc_candidates:
    raise SystemExit("PBS: couldn't find a 'Services' column.")

svc_col = svc_candidates[0]

# I build a real datetime month.
if {"Year", "Month"} <= cols:
    m = pbs_raw["Month"].astype(str).str.strip()
    y = pbs_raw["Year"].astype(str).str.strip()
    dt = pd.to_datetime(m + "-" + y, format="%B-%Y", errors="coerce")
    if dt.notna().sum() == 0:
        dt = pd.to_datetime(m + "-" + y, format="%b-%Y", errors="coerce")
    if dt.notna().sum() == 0:
        # If month is numeric
        dt = pd.to_datetime(dict(year=pd.to_numeric(y, errors="coerce"),
                                 month=pd.to_numeric(m, errors="coerce"),
                                 day=1), errors="coerce")
else:
    dt = pd.to_datetime(pbs_raw["Month"], errors="coerce")

df = pbs_raw.copy()
df["Month_dt"] = dt
df[svc_col] = pd.to_numeric(df[svc_col], errors="coerce")
df = df.dropna(subset=["Month_dt", svc_col])

# I try to get the ATC code and class name.
if "ATC Code" in df.columns:
    df["atc_code"] = df["ATC Code"].astype(str)
    df["class_name"] = df.get("ATC Level 4 Name", "Unknown").astype(str)
elif "ATC_Classification" in df.columns:
    text = df["ATC_Classification"].astype(str)
    # capture J01… anywhere (also handles 'J 01' or '(J01AA)')
    df["atc_code"] = text.str.extract(r'(?i)\b(J\s*0*1[A-Z0-9]{0,3})\b', expand=False)
    df["atc_code"] = df["atc_code"].str.replace(r"\s+", "", regex=True).str.upper()
    # class name = text without a leading code + dash/colon, if present
    df["class_name"] = text.str.replace(r'(?i)^\s*J\s*0*1[A-Z0-9]{0,3}\s*[-–:]\s*', '', regex=True)
else:
    # If I can’t find any ATC columns, I still keep going but label them as unknown.
    df["atc_code"] = "NA"
    df["class_name"] = "Unknown"

# I keep antibiotics if I can detect them (J01 or class name containing 'antibacter/antibiot').
abx_mask = (
    df["atc_code"].astype(str).str.replace(r"\s+", "", regex=True).str.upper().str.startswith("J01")
    | df["class_name"].str.contains("antibacter", case=False, na=False)
    | df["class_name"].str.contains("antibiot",   case=False, na=False)
)
df_abx = df[abx_mask].copy()

# I only keep 2015.
df_abx = df_abx[df_abx["Month_dt"].dt.year == 2015]

# If nothing matched antibiotics, I fall back to ALL PBS rows in 2015 (so I still get figures).
use_df = df_abx if len(df_abx) > 0 else df[df["Month_dt"].dt.year == 2015].copy()
if len(df_abx) == 0:
    print("[Note] Couldn’t confidently detect antibiotics (J01). Falling back to all PBS rows in 2015 for charts.")

# I build a clean table for plotting.
pbs = pd.DataFrame({
    "rx_month":  use_df["Month_dt"].dt.to_period("M").astype(str),
    "class_name": use_df["class_name"].astype(str),
    "scripts":   use_df[svc_col].fillna(0).astype("float64")
})

# Per-capita (scripts per 1,000 people)
pbs["scripts_per_1000"] = pbs["scripts"] / (erp_2015 / 1_000.0)
pbs["rx_month_dt"] = pd.to_datetime(pbs["rx_month"], errors="coerce")
pbs = pbs.dropna(subset=["rx_month_dt"]).copy()

# =========================
# 3) Visualisations I want
# =========================

# (a) Average Antibiotic Scripts per 1,000 People by Month (2015)
monthly_rate = (
    pbs.sort_values("rx_month_dt")
       .groupby(pbs["rx_month_dt"].dt.to_period("M"))["scripts_per_1000"]
       .sum()
)
plt.figure()
monthly_rate.plot(marker="o")
plt.title("Average Antibiotic Scripts per 1,000 People by Month (2015)")
plt.xlabel("Month"); plt.ylabel("Scripts per 1,000"); plt.xticks(rotation=45)
plt.tight_layout(); plt.savefig(FIG_DIR / "avg_scripts_per_1000_by_month_2015.png", dpi=150); plt.close()

# (b) Top 5 Antibiotic Classes in 2015 (Scripts/1,000)
top5 = (
    pbs.groupby("class_name")["scripts_per_1000"]
       .sum()
       .nlargest(5)
)
plt.figure()
top5.plot(kind="bar")
plt.title("Top 5 Antibiotic Classes in 2015 (Scripts/1,000)")
plt.ylabel("Total Scripts per 1,000 in 2015")
plt.tight_layout(); plt.savefig(FIG_DIR / "top5_antibiotic_classes_2015.png", dpi=150); plt.close()

# (c) Cumulative scripts per 1,000 (2015)
cumulative = monthly_rate.cumsum()
plt.figure()
cumulative.plot(marker="o")
plt.title("Cumulative Antibiotic Scripts per 1,000 (2015)")
plt.xlabel("Month"); plt.ylabel("Cumulative Scripts per 1,000"); plt.xticks(rotation=45)
plt.tight_layout(); plt.savefig(FIG_DIR / "cumulative_scripts_per_1000_2015.png", dpi=150); plt.close()

print("\nSaved cleaned CSVs in:", OUT_DIR)
print("Saved figures in:", FIG_DIR)